In [1]:
import coiled
import dask.distributed
import dask.dataframe as dd

## Cluster setup

In [2]:
cluster = coiled.Cluster(name="demo-repartition-cluster", n_workers=5)

Output()

Found software environment build


In [3]:
client = dask.distributed.Client(cluster)

## Resonably sized CSV files

In [26]:
d_7d_2000 = dd.read_csv(
    "s3://coiled-datasets/timeseries/7d/csv/2000/*.csv",
    storage_options={"anon": True, 'use_ssl': True})
d_7d_2001 = dd.read_csv(
    "s3://coiled-datasets/timeseries/7d/csv/2001/*.csv",
    storage_options={"anon": True, 'use_ssl': True})

In [27]:
ddf_7d = dd.concat([d_7d_2000, d_7d_2001])

In [28]:
len(ddf_7d)

62899200

In [29]:
ddf_7d.npartitions

104

In [30]:
%%time

ddf_7d["name"].nunique().compute()

CPU times: user 134 ms, sys: 19.9 ms, total: 154 ms
Wall time: 16.6 s


26

## Tiny CSV files

In [31]:
d_1h_2000 = dd.read_csv(
    "s3://coiled-datasets/timeseries/1h/csv/2000/*.csv",
    storage_options={"anon": True, 'use_ssl': True})
d_1h_2001 = dd.read_csv(
    "s3://coiled-datasets/timeseries/1h/csv/2001/*.csv",
    storage_options={"anon": True, 'use_ssl': True})

In [32]:
ddf_1h = dd.concat([d_1h_2000, d_1h_2001])

In [33]:
ddf_1h.npartitions

17496

In [34]:
%%time

ddf_1h["name"].nunique().compute()

CPU times: user 6.7 s, sys: 630 ms, total: 7.33 s
Wall time: 4min 16s


26

## Repartition tiny CSVs then count

In [35]:
%%time

repartitioned = ddf_1h.repartition(100)

CPU times: user 6.97 ms, sys: 7.53 ms, total: 14.5 ms
Wall time: 17.7 ms


In [36]:
%%time

repartitioned["name"].nunique().compute()

CPU times: user 5.53 s, sys: 565 ms, total: 6.1 s
Wall time: 3min 58s


26

### Warning: The following three cells only work for ppl with write permission to coiled-datasets (they're commented out cause most readers don't have those permissions)

In [ ]:
# %%time

# repartitioned.to_parquet("s3://coiled-datasets/tmp/repartitioned")

In [4]:
# len(
#     dd.read_parquet("s3://coiled-datasets/tmp/repartitioned", storage_options={"anon": True, 'use_ssl': True})
# )

62985600

In [5]:
# %%time

# ddf = dd.read_parquet("s3://coiled-datasets/tmp/repartitioned", storage_options={"anon": True, 'use_ssl': True})
# ddf["name"].nunique().compute()

CPU times: user 115 ms, sys: 11.6 ms, total: 126 ms
Wall time: 4.14 s


26

## Reasonably sized Parquet files

In [6]:
p_7d_2000 = dd.read_parquet(
    "s3://coiled-datasets/timeseries/7d/parquet/2000",
    storage_options={"anon": True, 'use_ssl': True})
p_7d_2001 = dd.read_parquet(
    "s3://coiled-datasets/timeseries/7d/parquet/2001",
    storage_options={"anon": True, 'use_ssl': True})

In [7]:
p_ddf_7d = dd.concat([p_7d_2000, p_7d_2001])

In [8]:
p_ddf_7d.npartitions

104

In [9]:
%%time

p_ddf_7d["name"].nunique().compute()

CPU times: user 56.4 ms, sys: 5.3 ms, total: 61.7 ms
Wall time: 9.7 s


26

## Show Dataset

In [10]:
p_7d_2000.head()

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1008,Dan,-0.259374,-0.118314
2000-01-01 00:00:01,987,Patricia,0.069601,0.755351
2000-01-01 00:00:02,980,Zelda,-0.281843,-0.510507
2000-01-01 00:00:03,1020,Ursula,-0.569904,0.523132
2000-01-01 00:00:04,967,Michael,-0.251460,0.810930
